In [2]:
import torch 

In [11]:
T = torch.rand((5,2,3,1,8))
#tensors can be of any shape in torch
print(T)

tensor([[[[[0.7119, 0.6279, 0.1930, 0.7386, 0.3676, 0.6129, 0.5556, 0.5599]],

          [[0.8923, 0.5695, 0.9922, 0.2585, 0.3055, 0.8037, 0.7164, 0.8240]],

          [[0.1494, 0.3997, 0.9167, 0.7235, 0.3855, 0.9394, 0.7659, 0.2134]]],


         [[[0.6122, 0.8779, 0.3864, 0.6881, 0.2117, 0.9595, 0.4316, 0.6140]],

          [[0.2266, 0.1357, 0.6260, 0.8139, 0.8025, 0.0456, 0.7742, 0.6268]],

          [[0.3385, 0.1099, 0.2835, 0.4758, 0.6905, 0.4603, 0.9638, 0.4982]]]],



        [[[[0.8696, 0.7884, 0.3562, 0.8750, 0.9478, 0.9789, 0.2254, 0.2384]],

          [[0.9566, 0.0481, 0.5150, 0.4613, 0.8827, 0.6050, 0.4009, 0.3346]],

          [[0.1678, 0.8758, 0.7838, 0.5718, 0.8089, 0.7214, 0.9390, 0.0506]]],


         [[[0.7544, 0.4244, 0.1998, 0.3370, 0.6617, 0.3630, 0.5426, 0.3742]],

          [[0.1352, 0.3322, 0.2969, 0.1964, 0.5656, 0.0695, 0.7811, 0.0730]],

          [[0.7536, 0.9003, 0.1899, 0.9563, 0.4342, 0.5585, 0.8518, 0.8401]]]],



        [[[[0.5663, 0.5128, 0.4946, 0.23

# Speechbrain implementation : 

No full complex activation functions for quat and complex, 
must apply split activation functions on the different components 

In [ ]:
from speechbrain.nnet.complex_networks.c_CNN import CConv1d, CConv2d

# to mimic the output of sftf 
# 4 examples, 20 time windows, 64 "features"
# also works for cconv2d so we can transform any networks 
# with quat and complex layers
T = torch.rand((4, 20, 64))

cnn_1d = CConv1d(input_shape = T.shape, out_channels=8, kernel_size=3)

out_tensor = cnn_1d(T)
print(out_tensor.shape)

torch.Size([4, 20, 16])


In [1]:
import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:46<00:00, 3.70MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


print(device)

cuda:0


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.220
[1,  4000] loss: 1.995
[1,  6000] loss: 1.758
[1,  8000] loss: 1.632
[1, 10000] loss: 1.560
[1, 12000] loss: 1.474
[2,  2000] loss: 1.414
[2,  4000] loss: 1.389
[2,  6000] loss: 1.355
[2,  8000] loss: 1.341
[2, 10000] loss: 1.297
[2, 12000] loss: 1.294
Finished Training


In [9]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [10]:
net = Net()
net.load_state_dict(torch.load(PATH, weights_only=True))

<All keys matched successfully>

In [11]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 52 %


# Toy example CNN

Now we implement a CNN with quaternion operations 

Torch doesn't support Quaternion operations so I'll do it
with speechbrain

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [1]:
from quaternion_layers import QuaternionConv, QuaternionLinear

ModuleNotFoundError: No module named 'quaternion_layers'

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # ok well permuted 
        inputs = inputs.permute(0,2,3,1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')